In [14]:
import pandas as pd
import numpy as np

In [15]:
act24_ground = pd.read_csv("C:/Users/HELIOS-300/Desktop/Data/act24_prelim_groundtruth(in) (1).csv")
act24_ground.head()

C:\Users\HELIOS-300\AppData\Local\Temp\ipykernel_15704\3939913406.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  act24_ground = pd.read_csv("C:/Users/HELIOS-300/Desktop/Data/act24_prelim_groundtruth(in) (1).csv")


,id,observation,date,date_time,relative_time,activity_type,broad_activity_type,work_type,posture,sedentary_not,walking_not,activity_intensity,quality,step
0,102,1,7/24/2019,7/24/2019 8:20,0:00:00,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,Codable,0
1,102,1,7/24/2019,7/24/2019 8:20,0:00:01,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,Codable,0
2,102,1,7/24/2019,7/24/2019 8:20,0:00:02,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,Codable,0
3,102,1,7/24/2019,7/24/2019 8:20,0:00:03,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,Codable,0
4,102,1,7/24/2019,7/24/2019 8:20,0:00:04,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light,Codable,0


In [16]:
act24_cam = pd.read_csv("C:/Users/HELIOS-300/Desktop/WAVES/ACT24 Full Code/Cameron_ACT24_Clean.csv")
act24_cam = act24_cam[["id", "obs", "date", "date_time", "rel_time", "activity_type", "broad_domain", "waves_domain", "posture_wbm", "intensity"]]

act24_cam = act24_cam.rename(columns={
    "obs" : "observation",
    "rel_time" : "relative_time",
    "broad_domain" : "broad_activity_type",
    "waves_domain" : "work_type",
    "posture_wbm" : "posture",
    "intensity" : "activity_intensity"
})

act24_cam.head()

C:\Users\HELIOS-300\AppData\Local\Temp\ipykernel_15704\2165436145.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  act24_cam = pd.read_csv("C:/Users/HELIOS-300/Desktop/WAVES/ACT24 Full Code/Cameron_ACT24_Clean.csv")


,id,observation,date,date_time,relative_time,activity_type,broad_activity_type,work_type,posture,activity_intensity
0,102,1,7/24/2019,7/24/2019 08:21:12 AM,00:00:00,work_general,work_education,work_education,stand,light
1,102,1,7/24/2019,7/24/2019 08:21:12 AM,00:00:01,work_general,work_education,work_education,stand,light
2,102,1,7/24/2019,7/24/2019 08:21:12 AM,00:00:02,work_general,work_education,work_education,stand,light
3,102,1,7/24/2019,7/24/2019 08:21:12 AM,00:00:03,work_general,work_education,work_education,stand,light
4,102,1,7/24/2019,7/24/2019 08:21:12 AM,00:00:04,work_general,work_education,work_education,stand,light


In [17]:
print("ground shape: ", act24_ground.shape)
print("cam shape: ", act24_cam.shape)

ground shape:  (401689, 14)
cam shape:  (460935, 10)


In [18]:
# COMPREHENSIVE COMPARISON OF SECOND-BY-SECOND DATA
# These are second-by-second files where each row represents one second
# Merge key: id + observation + relative_time (should be unique for each second)

print("=" * 80)
print("COMPREHENSIVE COMPARISON: act24_ground vs act24_cam")
print("=" * 80)

# Find common columns to compare
common_cols = set(act24_ground.columns) & set(act24_cam.columns)
merge_keys = ['id', 'observation', 'relative_time']
compare_cols = [c for c in common_cols if c not in merge_keys]

print(f"\nCommon columns to compare ({len(compare_cols)}): {sorted(compare_cols)}")
print(f"Merge keys: {merge_keys}")

# Normalize relative_time format for comparison (handle potential format differences)
# Convert relative_time to consistent format (HH:MM:SS)
def normalize_time(time_str):
    if pd.isna(time_str):
        return None
    time_str = str(time_str).strip()
    # Handle formats like "0:00:00", "00:00:00", "0:00:25.01", etc.
    parts = time_str.split(':')
    if len(parts) == 3:
        # Handle decimal seconds (e.g., "25.01" -> "25") by converting to float then int
        seconds_part = str(int(float(parts[2])))
        return f"{int(parts[0]):02d}:{int(parts[1]):02d}:{seconds_part.zfill(2)}"
    return time_str

act24_ground['_rel_time_norm'] = act24_ground['relative_time'].apply(normalize_time)
act24_cam['_rel_time_norm'] = act24_cam['relative_time'].apply(normalize_time)

# Create composite merge key: id|observation|relative_time
act24_ground['_merge_key'] = (
    act24_ground['id'].astype(str) + '|' + 
    act24_ground['observation'].astype(str) + '|' + 
    act24_ground['_rel_time_norm'].astype(str)
)
act24_cam['_merge_key'] = (
    act24_cam['id'].astype(str) + '|' + 
    act24_cam['observation'].astype(str) + '|' + 
    act24_cam['_rel_time_norm'].astype(str)
)

# Find overlapping keys
ground_keys = set(act24_ground['_merge_key'].dropna())
cam_keys = set(act24_cam['_merge_key'].dropna())
common_keys = ground_keys & cam_keys

print(f"\nUnique keys in ground: {len(ground_keys):,}")
print(f"Unique keys in cam: {len(cam_keys):,}")
print(f"Overlapping keys (rows in both): {len(common_keys):,}")

# Compare each common column
print(f"\n{'='*80}")
print(f"COLUMN-BY-COLUMN COMPARISON")
print(f"{'='*80}")

comparison_results = []

for col in compare_cols:
    if col not in act24_ground.columns or col not in act24_cam.columns:
        continue
    
    print(f"\n{col}:")
    
    # Filter to common keys and merge
    ground_subset = act24_ground[act24_ground['_merge_key'].isin(common_keys)][['_merge_key', col]].copy()
    cam_subset = act24_cam[act24_cam['_merge_key'].isin(common_keys)][['_merge_key', col]].copy()
    
    # Merge on the key
    merged = ground_subset.merge(
        cam_subset,
        on='_merge_key',
        how='inner',
        suffixes=('_ground', '_cam')
    )
    
    if len(merged) > 0:
        # Compare values
        merged['_match'] = (merged[f'{col}_ground'].astype(str) == merged[f'{col}_cam'].astype(str))
        match_count = merged['_match'].sum()
        mismatch_count = (~merged['_match']).sum()
        match_pct = (match_count / len(merged)) * 100
        
        comparison_results.append({
            'column': col,
            'rows_compared': len(merged),
            'exact_matches': match_count,
            'match_percentage': match_pct,
            'mismatches': mismatch_count,
            'mismatch_percentage': 100 - match_pct
        })
        
        print(f"  Rows compared: {len(merged):,}")
        print(f"  Exact matches: {match_count:,} ({match_pct:.2f}%)")
        print(f"  Mismatches: {mismatch_count:,} ({100-match_pct:.2f}%)")
        
        # Show sample mismatches
        if mismatch_count > 0:
            mismatches = merged[~merged['_match']].head(5)
            print(f"  Sample mismatches:")
            for idx, row in mismatches.iterrows():
                key_parts = row['_merge_key'].split('|')
                print(f"    id={key_parts[0]}, obs={key_parts[1]}, time={key_parts[2]}")
                print(f"      Ground: {row[f'{col}_ground']}")
                print(f"      Cam: {row[f'{col}_cam']}")
        
        # Value distribution comparison
        ground_vals = set(merged[f'{col}_ground'].dropna().astype(str))
        cam_vals = set(merged[f'{col}_cam'].dropna().astype(str))
        only_ground = ground_vals - cam_vals
        only_cam = cam_vals - ground_vals
        
        if len(only_ground) > 0:
            print(f"  Values only in ground ({len(only_ground)}): {list(only_ground)[:5]}")
        if len(only_cam) > 0:
            print(f"  Values only in cam ({len(only_cam)}): {list(only_cam)[:5]}")
    else:
        print(f"  No overlapping rows to compare")

# Clean up temporary columns
act24_ground.drop(columns=['_rel_time_norm', '_merge_key'], inplace=True, errors='ignore')
act24_cam.drop(columns=['_rel_time_norm', '_merge_key'], inplace=True, errors='ignore')

# Summary
if comparison_results:
    comparison_summary = pd.DataFrame(comparison_results)
    print(f"\n{'='*80}")
    print("COMPARISON SUMMARY")
    print(f"{'='*80}")
    print(comparison_summary.to_string(index=False))


COMPREHENSIVE COMPARISON: act24_ground vs act24_cam

Common columns to compare (7): ['activity_intensity', 'activity_type', 'broad_activity_type', 'date', 'date_time', 'posture', 'work_type']
Merge keys: ['id', 'observation', 'relative_time']

Unique keys in ground: 401,689
Unique keys in cam: 415,556
Overlapping keys (rows in both): 335,157

COLUMN-BY-COLUMN COMPARISON

work_type:
  Rows compared: 367,487
  Exact matches: 0 (0.00%)
  Mismatches: 367,487 (100.00%)
  Sample mismatches:
    id=102, obs=1, time=00:00:00
      Ground: SP- Education and Health Services
      Cam: work_education
    id=102, obs=1, time=00:00:01
      Ground: SP- Education and Health Services
      Cam: work_education
    id=102, obs=1, time=00:00:02
      Ground: SP- Education and Health Services
      Cam: work_education
    id=102, obs=1, time=00:00:03
      Ground: SP- Education and Health Services
      Cam: work_education
    id=102, obs=1, time=00:00:04
      Ground: SP- Education and Health Services
 

In [19]:
# COMPREHENSIVE ANALYSIS: ROWS THAT EXIST IN ONE DATAFRAME BUT NOT THE OTHER
# This analysis explains why the dataframes have different row counts and provides
# detailed breakdowns to understand the differences

print("=" * 80)
print("COMPREHENSIVE ROW COUNT DIFFERENCES ANALYSIS")
print("=" * 80)

print(f"\nOVERVIEW:")
print(f"  act24_ground total rows: {len(act24_ground):,}")
print(f"  act24_cam total rows: {len(act24_cam):,}")
print(f"  Absolute difference: {abs(len(act24_cam) - len(act24_ground)):,} rows")
print(f"  Percentage difference: {abs(len(act24_cam) - len(act24_ground)) / len(act24_ground) * 100:.2f}%")
if len(act24_cam) > len(act24_ground):
    print(f"  → act24_cam has {len(act24_cam) - len(act24_ground):,} MORE rows than act24_ground")
else:
    print(f"  → act24_ground has {len(act24_ground) - len(act24_cam):,} MORE rows than act24_cam")

# Helper functions
def normalize_time(time_str):
    if pd.isna(time_str):
        return None
    time_str = str(time_str).strip()
    parts = time_str.split(':')
    if len(parts) == 3:
        seconds_part = str(int(float(parts[2])))
        return f"{int(parts[0]):02d}:{int(parts[1]):02d}:{seconds_part.zfill(2)}"
    return time_str

def time_to_seconds(time_str):
    if pd.isna(time_str):
        return None
    time_str = str(time_str).strip()
    parts = time_str.split(':')
    if len(parts) == 3:
        return int(parts[0]) * 3600 + int(parts[1]) * 60 + int(float(parts[2]))
    return None

act24_ground['_rel_time_norm'] = act24_ground['relative_time'].apply(normalize_time)
act24_cam['_rel_time_norm'] = act24_cam['relative_time'].apply(normalize_time)

act24_ground['_merge_key'] = (
    act24_ground['id'].astype(str) + '|' + 
    act24_ground['observation'].astype(str) + '|' + 
    act24_ground['_rel_time_norm'].astype(str)
)
act24_cam['_merge_key'] = (
    act24_cam['id'].astype(str) + '|' + 
    act24_cam['observation'].astype(str) + '|' + 
    act24_cam['_rel_time_norm'].astype(str)
)

# Find keys in each dataframe
ground_keys = set(act24_ground['_merge_key'].dropna())
cam_keys = set(act24_cam['_merge_key'].dropna())
common_keys = ground_keys & cam_keys
only_ground = ground_keys - cam_keys
only_cam = cam_keys - ground_keys

print(f"\n{'='*80}")
print("KEY OVERLAP ANALYSIS")
print(f"{'='*80}")
print(f"\nEach row is uniquely identified by: id + observation + relative_time")
print(f"\nUnique row identifiers (keys):")
print(f"  Total unique keys in ground: {len(ground_keys):,}")
print(f"  Total unique keys in cam: {len(cam_keys):,}")
print(f"  Keys present in BOTH dataframes: {len(common_keys):,} ({len(common_keys)/max(len(ground_keys), len(cam_keys))*100:.1f}% of larger dataset)")
print(f"  Keys ONLY in ground: {len(only_ground):,} ({len(only_ground)/len(ground_keys)*100:.1f}% of ground)")
print(f"  Keys ONLY in cam: {len(only_cam):,} ({len(only_cam)/len(cam_keys)*100:.1f}% of cam)")

print(f"\nInterpretation:")
print(f"  • {len(common_keys):,} seconds are recorded in BOTH datasets (can be compared)")
print(f"  • {len(only_ground):,} seconds exist ONLY in ground dataset (missing from cam)")
print(f"  • {len(only_cam):,} seconds exist ONLY in cam dataset (missing from ground)")
print(f"  • Net difference: {len(only_cam) - len(only_ground):,} more unique seconds in cam")

# Analyze which id/observation combinations have differences
print(f"\n{'='*80}")
print("DETAILED BREAKDOWN BY ID AND OBSERVATION")
print(f"{'='*80}")

# Get unique id/observation combinations
ground_id_obs = act24_ground[['id', 'observation']].drop_duplicates()
cam_id_obs = act24_cam[['id', 'observation']].drop_duplicates()

print(f"\nUnique (id, observation) combinations:")
print(f"  In ground: {len(ground_id_obs)} combinations")
print(f"  In cam: {len(cam_id_obs)} combinations")
print(f"  In both: {len(set(zip(ground_id_obs['id'], ground_id_obs['observation'])) & set(zip(cam_id_obs['id'], cam_id_obs['observation'])))} combinations")
only_ground_combos = set(zip(ground_id_obs['id'], ground_id_obs['observation'])) - set(zip(cam_id_obs['id'], cam_id_obs['observation']))
only_cam_combos = set(zip(cam_id_obs['id'], cam_id_obs['observation'])) - set(zip(ground_id_obs['id'], ground_id_obs['observation']))
print(f"  Only in ground: {len(only_ground_combos)} combinations")
print(f"  Only in cam: {len(only_cam_combos)} combinations")

# Count rows per id/observation
ground_counts = act24_ground.groupby(['id', 'observation']).size().reset_index(name='ground_count')
cam_counts = act24_cam.groupby(['id', 'observation']).size().reset_index(name='cam_count')

# Merge to compare
counts_merged = ground_counts.merge(
    cam_counts,
    on=['id', 'observation'],
    how='outer'
)

# Fill NaN values with 0 (for combinations that only exist in one dataframe)
counts_merged['ground_count'] = counts_merged['ground_count'].fillna(0).astype(int)
counts_merged['cam_count'] = counts_merged['cam_count'].fillna(0).astype(int)
counts_merged['difference'] = counts_merged['cam_count'] - counts_merged['ground_count']
counts_merged['abs_difference'] = counts_merged['difference'].abs()
counts_merged['percent_diff'] = (counts_merged['difference'] / counts_merged[['ground_count', 'cam_count']].max(axis=1) * 100).round(2)

# Sort by absolute difference
counts_merged_sorted = counts_merged.sort_values('abs_difference', ascending=False)

print(f"\n{'='*80}")
print("ALL (ID, OBSERVATION) COMBINATIONS - COMPLETE LIST")
print(f"{'='*80}")
print(f"\nShowing ALL {len(counts_merged)} combinations, sorted by absolute difference:")
print(counts_merged_sorted[['id', 'observation', 'ground_count', 'cam_count', 'difference', 'percent_diff']].to_string(index=False))

# Summary statistics
print(f"\n{'='*80}")
print("SUMMARY STATISTICS")
print(f"{'='*80}")

cam_more = counts_merged[counts_merged['difference'] > 0]
ground_more = counts_merged[counts_merged['difference'] < 0]
equal = counts_merged[counts_merged['difference'] == 0]

print(f"\nCombinations where cam has MORE rows:")
print(f"  Count: {len(cam_more)} combinations")
print(f"  Total extra rows: {cam_more['difference'].sum():,}")
print(f"  Average extra rows per combination: {cam_more['difference'].mean():.1f}")
print(f"  Median extra rows: {cam_more['difference'].median():.0f}")
print(f"  Max extra rows: {cam_more['difference'].max():,}")

print(f"\nCombinations where ground has MORE rows:")
print(f"  Count: {len(ground_more)} combinations")
print(f"  Total extra rows: {abs(ground_more['difference'].sum()):,}")
print(f"  Average extra rows per combination: {abs(ground_more['difference']).mean():.1f}")
print(f"  Median extra rows: {abs(ground_more['difference']).median():.0f}")
print(f"  Max extra rows: {abs(ground_more['difference']).max():,}")

print(f"\nCombinations with EQUAL row counts:")
print(f"  Count: {len(equal)} combinations")

# Detailed breakdown of rows only in ground
if len(only_ground) > 0:
    print(f"\n{'='*80}")
    print("ROWS ONLY IN GROUND - DETAILED ANALYSIS")
    print(f"{'='*80}")
    
    ground_only_df = act24_ground[act24_ground['_merge_key'].isin(only_ground)].copy()
    
    print(f"\nTotal rows only in ground: {len(ground_only_df):,}")
    
    # Breakdown by id/observation
    print(f"\nBreakdown by (id, observation) - ALL combinations:")
    ground_only_by_id = ground_only_df.groupby(['id', 'observation']).size().reset_index(name='count')
    ground_only_by_id = ground_only_by_id.sort_values('count', ascending=False)
    print(ground_only_by_id.to_string(index=False))
    
    # Breakdown by date
    print(f"\nBreakdown by date:")
    ground_only_by_date = ground_only_df.groupby('date').size().reset_index(name='count')
    ground_only_by_date = ground_only_by_date.sort_values('count', ascending=False)
    print(ground_only_by_date.to_string(index=False))
    
    # Sample rows with more context
    print(f"\nSample rows (first 20, showing full context):")
    sample_ground = ground_only_df.head(20)
    print(sample_ground[['id', 'observation', 'date', 'relative_time', 'activity_type', 'posture', 'activity_intensity']].to_string(index=False))
    
    # Time distribution
    ground_only_times = ground_only_df['relative_time'].apply(time_to_seconds)
    print(f"\nTime distribution for rows only in ground:")
    print(f"  Min time: {ground_only_times.min()} seconds ({ground_only_times.min() // 3600}:{(ground_only_times.min() % 3600) // 60:02d}:{ground_only_times.min() % 60:02d})")
    print(f"  Max time: {ground_only_times.max()} seconds ({ground_only_times.max() // 3600}:{(ground_only_times.max() % 3600) // 60:02d}:{ground_only_times.max() % 60:02d})")
    print(f"  Mean time: {ground_only_times.mean():.0f} seconds ({int(ground_only_times.mean()) // 3600}:{(int(ground_only_times.mean()) % 3600) // 60:02d}:{int(ground_only_times.mean()) % 60:02d})")
    print(f"  Median time: {ground_only_times.median():.0f} seconds ({int(ground_only_times.median()) // 3600}:{(int(ground_only_times.median()) % 3600) // 60:02d}:{int(ground_only_times.median()) % 60:02d})")

# Detailed breakdown of rows only in cam
if len(only_cam) > 0:
    print(f"\n{'='*80}")
    print("ROWS ONLY IN CAM - DETAILED ANALYSIS")
    print(f"{'='*80}")
    
    cam_only_df = act24_cam[act24_cam['_merge_key'].isin(only_cam)].copy()
    
    print(f"\nTotal rows only in cam: {len(cam_only_df):,}")
    
    # Breakdown by id/observation
    print(f"\nBreakdown by (id, observation) - ALL combinations:")
    cam_only_by_id = cam_only_df.groupby(['id', 'observation']).size().reset_index(name='count')
    cam_only_by_id = cam_only_by_id.sort_values('count', ascending=False)
    print(cam_only_by_id.to_string(index=False))
    
    # Breakdown by date
    print(f"\nBreakdown by date:")
    cam_only_by_date = cam_only_df.groupby('date').size().reset_index(name='count')
    cam_only_by_date = cam_only_by_date.sort_values('count', ascending=False)
    print(cam_only_by_date.to_string(index=False))
    
    # Sample rows with more context
    print(f"\nSample rows (first 20, showing full context):")
    sample_cam = cam_only_df.head(20)
    print(sample_cam[['id', 'observation', 'date', 'relative_time', 'activity_type', 'posture', 'activity_intensity']].to_string(index=False))
    
    # Time distribution
    cam_only_times = cam_only_df['relative_time'].apply(time_to_seconds)
    print(f"\nTime distribution for rows only in cam:")
    print(f"  Min time: {cam_only_times.min()} seconds ({cam_only_times.min() // 3600}:{(cam_only_times.min() % 3600) // 60:02d}:{cam_only_times.min() % 60:02d})")
    print(f"  Max time: {cam_only_times.max()} seconds ({cam_only_times.max() // 3600}:{(cam_only_times.max() % 3600) // 60:02d}:{cam_only_times.max() % 60:02d})")
    print(f"  Mean time: {cam_only_times.mean():.0f} seconds ({int(cam_only_times.mean()) // 3600}:{(int(cam_only_times.mean()) % 3600) // 60:02d}:{int(cam_only_times.mean()) % 60:02d})")
    print(f"  Median time: {cam_only_times.median():.0f} seconds ({int(cam_only_times.median()) // 3600}:{(int(cam_only_times.median()) % 3600) // 60:02d}:{int(cam_only_times.median()) % 60:02d})")

# Final summary
print(f"\n{'='*80}")
print("FINAL SUMMARY AND INTERPRETATION")
print(f"{'='*80}")
print(f"\nKey Findings:")
print(f"  1. Row count difference: {len(act24_cam) - len(act24_ground):,} rows ({abs(len(act24_cam) - len(act24_ground)) / len(act24_ground) * 100:.1f}%)")
print(f"  2. Overlapping data: {len(common_keys):,} seconds appear in both datasets")
print(f"  3. Missing from cam: {len(only_ground):,} seconds in ground are not in cam")
print(f"  4. Missing from ground: {len(only_cam):,} seconds in cam are not in ground")
print(f"  5. Unique combinations: {len(only_cam_combos)} (id, obs) pairs exist only in cam")
print(f"  6. Unique combinations: {len(only_ground_combos)} (id, obs) pairs exist only in ground")
print(f"\nThis suggests:")
print(f"  • Different data collection/processing methods may have included/excluded certain time periods")
print(f"  • Some (id, observation) combinations may have been processed differently")
print(f"  • Time range differences may indicate different start/end times or filtering criteria")

# Clean up temporary columns
act24_ground.drop(columns=['_rel_time_norm', '_merge_key'], inplace=True, errors='ignore')
act24_cam.drop(columns=['_rel_time_norm', '_merge_key'], inplace=True, errors='ignore')


COMPREHENSIVE ROW COUNT DIFFERENCES ANALYSIS

OVERVIEW:
  act24_ground total rows: 401,689
  act24_cam total rows: 460,935
  Absolute difference: 59,246 rows
  Percentage difference: 14.75%
  → act24_cam has 59,246 MORE rows than act24_ground

KEY OVERLAP ANALYSIS

Each row is uniquely identified by: id + observation + relative_time

Unique row identifiers (keys):
  Total unique keys in ground: 401,689
  Total unique keys in cam: 415,556
  Keys present in BOTH dataframes: 335,157 (80.7% of larger dataset)
  Keys ONLY in ground: 66,532 (16.6% of ground)
  Keys ONLY in cam: 80,399 (19.3% of cam)

Interpretation:
  • 335,157 seconds are recorded in BOTH datasets (can be compared)
  • 66,532 seconds exist ONLY in ground dataset (missing from cam)
  • 80,399 seconds exist ONLY in cam dataset (missing from ground)
  • Net difference: 13,867 more unique seconds in cam

DETAILED BREAKDOWN BY ID AND OBSERVATION

Unique (id, observation) combinations:
  In ground: 40 combinations
  In cam: 48 co

In [20]:
test1 = act24_ground[act24_ground["id"] == 154]
test1 = act24_ground[act24_ground["observation"] == 2]
test1.shape

ACT24_CAM DATAFRAME EXPLORATION

Shape: (460935, 10)

Column names (10):
['id', 'observation', 'date', 'date_time', 'relative_time', 'activity_type', 'broad_activity_type', 'work_type', 'posture', 'activity_intensity']

Data types:
id                      int64
observation             int64
date                   object
date_time              object
relative_time          object
activity_type          object
broad_activity_type    object
work_type              object
posture                object
activity_intensity     object
dtype: object

First few rows:
    id  observation       date              date_time relative_time  \
0  102            1  7/24/2019  7/24/2019 08:21:12 AM      00:00:00   
1  102            1  7/24/2019  7/24/2019 08:21:12 AM      00:00:01   
2  102            1  7/24/2019  7/24/2019 08:21:12 AM      00:00:02   
3  102            1  7/24/2019  7/24/2019 08:21:12 AM      00:00:03   
4  102            1  7/24/2019  7/24/2019 08:21:12 AM      00:00:04   

  activity

In [23]:
# FOCUSED REPORT: (ID, OBSERVATION) PAIRS IN ONE DATASET BUT NOT THE OTHER
print("=" * 80)
print("FOCUSED REPORT: MISSING (ID, OBSERVATION) PAIRS")
print("=" * 80)

# Get all unique (id, observation) combinations
ground_id_obs = act24_ground[['id', 'observation']].drop_duplicates()
cam_id_obs = act24_cam[['id', 'observation']].drop_duplicates()

ground_combos = set(zip(ground_id_obs['id'], ground_id_obs['observation']))
cam_combos = set(zip(cam_id_obs['id'], cam_id_obs['observation']))
common_combos = ground_combos & cam_combos
only_ground_combos = ground_combos - cam_combos
only_cam_combos = cam_combos - ground_combos

# Get row counts for context
ground_counts = act24_ground.groupby(['id', 'observation']).size().reset_index(name='count')
cam_counts = act24_cam.groupby(['id', 'observation']).size().reset_index(name='count')

print("\n" + "=" * 80)
print("SUMMARY")
print("=" * 80)
print(f"Total (ID, Observation) pairs in ground: {len(ground_combos)}")
print(f"Total (ID, Observation) pairs in cam: {len(cam_combos)}")
print(f"Pairs in BOTH datasets: {len(common_combos)}")
print(f"Pairs ONLY in ground: {len(only_ground_combos)}")
print(f"Pairs ONLY in cam: {len(only_cam_combos)}")

print("\n" + "=" * 80)
print("(ID, OBSERVATION) PAIRS ONLY IN GROUND TRUTH")
print("=" * 80)
if len(only_ground_combos) == 0:
    print("None - All pairs in ground truth are also in cam dataset")
else:
    only_ground_sorted = sorted(only_ground_combos)
    print(f"\n{len(only_ground_combos)} pair(s) missing from cam dataset:\n")
    for id_val, obs_val in only_ground_sorted:
        count = ground_counts[(ground_counts['id'] == id_val) & (ground_counts['observation'] == obs_val)]['count'].values[0]
        date_info = act24_ground[(act24_ground['id'] == id_val) & (act24_ground['observation'] == obs_val)]['date'].iloc[0] if len(act24_ground[(act24_ground['id'] == id_val) & (act24_ground['observation'] == obs_val)]) > 0 else "N/A"
        print(f"  ID {id_val:3d}, Observation {obs_val:1d}: {count:6,} rows | Date: {date_info}")

print("\n" + "=" * 80)
print("(ID, OBSERVATION) PAIRS ONLY IN CAM DATASET")
print("=" * 80)
if len(only_cam_combos) == 0:
    print("None - All pairs in cam dataset are also in ground truth")
else:
    only_cam_sorted = sorted(only_cam_combos)
    print(f"\n{len(only_cam_combos)} pair(s) missing from ground truth:\n")
    for id_val, obs_val in only_cam_sorted:
        count = cam_counts[(cam_counts['id'] == id_val) & (cam_counts['observation'] == obs_val)]['count'].values[0]
        date_info = act24_cam[(act24_cam['id'] == id_val) & (act24_cam['observation'] == obs_val)]['date'].iloc[0] if len(act24_cam[(act24_cam['id'] == id_val) & (act24_cam['observation'] == obs_val)]) > 0 else "N/A"
        print(f"  ID {id_val:3d}, Observation {obs_val:1d}: {count:6,} rows | Date: {date_info}")

print("\n" + "=" * 80)
print("COMPLETE (ID, OBSERVATION) PAIR COMPARISON TABLE")
print("=" * 80)
# Create comprehensive comparison table
all_combos = ground_combos | cam_combos
comparison_data = []
for id_val, obs_val in sorted(all_combos):
    in_ground = (id_val, obs_val) in ground_combos
    in_cam = (id_val, obs_val) in cam_combos
    
    ground_count = ground_counts[(ground_counts['id'] == id_val) & (ground_counts['observation'] == obs_val)]['count'].values[0] if in_ground else 0
    cam_count = cam_counts[(cam_counts['id'] == id_val) & (cam_counts['observation'] == obs_val)]['count'].values[0] if in_cam else 0
    
    status = "Both" if (in_ground and in_cam) else ("Ground only" if in_ground else "Cam only")
    
    comparison_data.append({
        'ID': id_val,
        'Observation': obs_val,
        'In Ground': 'Yes' if in_ground else 'No',
        'In Cam': 'Yes' if in_cam else 'No',
        'Ground Rows': ground_count,
        'Cam Rows': cam_count,
        'Difference': cam_count - ground_count,
        'Status': status
    })

comparison_df = pd.DataFrame(comparison_data)
print("\n" + comparison_df.to_string(index=False))

print("\n" + "=" * 80)
print("KEY FINDINGS")
print("=" * 80)
print(f"• {len(only_ground_combos)} (ID, Observation) pair(s) exist in ground truth but NOT in your cam dataset")
print(f"• {len(only_cam_combos)} (ID, Observation) pair(s) exist in your cam dataset but NOT in ground truth")
print(f"• {len(common_combos)} (ID, Observation) pair(s) exist in both datasets")
if len(only_ground_combos) > 0:
    print(f"\n⚠️  Your cam dataset is missing {len(only_ground_combos)} observation(s) from ground truth")
if len(only_cam_combos) > 0:
    print(f"⚠️  Your cam dataset includes {len(only_cam_combos)} observation(s) not in ground truth")


FOCUSED REPORT: MISSING (ID, OBSERVATION) PAIRS

SUMMARY
Total (ID, Observation) pairs in ground: 40
Total (ID, Observation) pairs in cam: 48
Pairs in BOTH datasets: 40
Pairs ONLY in ground: 0
Pairs ONLY in cam: 8

(ID, OBSERVATION) PAIRS ONLY IN GROUND TRUTH
None - All pairs in ground truth are also in cam dataset

(ID, OBSERVATION) PAIRS ONLY IN CAM DATASET

8 pair(s) missing from ground truth:

  ID 126, Observation 1: 10,394 rows | Date: 9/2/2019
  ID 131, Observation 2: 10,107 rows | Date: 9/16/2019
  ID 134, Observation 2: 10,726 rows | Date: 10/13/2019
  ID 136, Observation 2: 10,364 rows | Date: 10/19/2019
  ID 138, Observation 2:  9,925 rows | Date: 11/2/2019
  ID 139, Observation 1: 10,584 rows | Date: 11/3/2019
  ID 139, Observation 2:  7,458 rows | Date: 11/5/2019
  ID 154, Observation 1: 10,615 rows | Date: 2/22/2020

COMPLETE (ID, OBSERVATION) PAIR COMPARISON TABLE

 ID  Observation In Ground In Cam  Ground Rows  Cam Rows  Difference   Status
102            1       Yes   

In [22]:
test2 = act24_cam[act24_cam["id"] == 154]
test2 = act24_cam[act24_cam["observation"] == 2]
test2.shape

(229561, 10)